## Cleaning file for 2019-2020

In [1]:
import pandas as pd
import numpy as np

In [2]:
roster_data_1920 = pd.read_stata('/Users/preetkhowaja/Documents/midssp2022/unifying/uds-2022-ids-701-team-3/00_source_data/1920_roster.dta')
roster_data_1920.head()

,hhcode,psu,province,region,district,idc,sb1q2,sb1q3,sb1q4,sb1q5,age,sb1q62,sb1q63,sb1q64,sb1q7,sb1q8,sb1q9,sb1q10,sb1q11
0,101100101,1011001,khyber pakhtunkhwa,rural,abbottabad,1,head,NaN,male,present,58,0,0,1961.0,currently married,2.0,98.0,99.0,yes
1,101100102,1011001,khyber pakhtunkhwa,rural,abbottabad,1,head,NaN,male,present,64,0,0,1955.0,currently married,2.0,98.0,10.0,yes
2,101100103,1011001,khyber pakhtunkhwa,rural,abbottabad,1,head,NaN,male,present,35,0,0,1984.0,currently married,2.0,98.0,99.0,yes
3,101100104,1011001,khyber pakhtunkhwa,rural,abbottabad,1,head,NaN,male,present,56,0,0,1963.0,currently married,2.0,98.0,99.0,yes
4,101100105,1011001,khyber pakhtunkhwa,rural,abbottabad,1,head,NaN,male,present,50,0,0,1969.0,currently married,2.0,98.0,98.0,yes


In [3]:
# renaming relevant variables from roster
roster_data_1920['sex'] = roster_data_1920['sb1q4']
roster_data_1920['marital_status'] = roster_data_1920['sb1q7']
roster_data_1920.sample(4)

,hhcode,psu,province,region,district,idc,sb1q2,sb1q3,sb1q4,sb1q5,...,sb1q62,sb1q63,sb1q64,sb1q7,sb1q8,sb1q9,sb1q10,sb1q11,sex,marital_status
71480,220102011,2201020,punjab,rural,lodhran,1,head,NaN,male,present,...,0,0,1962.0,currently married,2.0,98.0,98.0,yes,male,currently married
426261,307211502,3072115,sindh,urban,karachi central,3,son/daughter,main economic provider,male,present,...,0,0,2007.0,unmarried / never married,NaN,1.0,2.0,yes,male,unmarried / never married
443524,321200402,3212004,sindh,urban,shaheed banazir abad,3,son/daughter,main economic provider,male,present,...,0,0,1998.0,unmarried / never married,NaN,1.0,2.0,yes,male,unmarried / never married
734876,231207314,2312073,punjab,urban,rawalpindi,6,son/daughter,main economic provider,female,present,...,0,0,2003.0,unmarried / never married,NaN,1.0,2.0,yes,female,unmarried / never married


In [4]:
B_relevant = roster_data_1920[['hhcode', 'idc','age', 'sex', 'marital_status']].copy()
B_relevant.sample(5)

,hhcode,idc,age,sex,marital_status
202799,208103022,2,33,female,currently married
408533,233101806,3,12,male,unmarried / never married
95812,232100705,1,58,male,currently married
435865,314202829,3,5,female,unmarried / never married
33292,203101808,1,30,male,currently married


In [5]:
educ_data = pd.read_stata('/Users/preetkhowaja/Documents/midssp2022/unifying/uds-2022-ids-701-team-3/00_source_data/1920_secc1.dta',
convert_categoricals=False)
educ_data.columns

Index(['hhcode', 'psu', 'province', 'region', 'idc', 'district', 'sc1q1a',
       'sc1q2a', 'sc1q3a', 'sc1q4a', 'sc1q01', 'sc1q02', 'sc1q03', 'sc1q04',
       'sc1q05', 'sc1q06', 'sc1q07', 'sc1q08', 'sc1q09', 'sc1q10', 'sc1q11',
       'sc1q12', 'sc1q13', 'sc1q14', 'sc1q15', 'sc1q16', 'sc1q17', 'sc1q18',
       'sc1q19a', 'sc1q19b', 'sc1q19c', 'sc1q19d', 'sc1q19e', 'sc1q19f',
       'sc1q19g', 'sc1q19h', 'sc1q19i'],
      dtype='object')

In [6]:
educ_data.sc1q01.value_counts()

1    355074
2    244505
3    214697
Name: sc1q01, dtype: int64

In [7]:
# Create variable for ever_admitted and currently attending
educ_data['ever_admitted'] = np.where(educ_data.sc1q01 == 1, 'no', 'yes')
educ_data.ever_admitted.value_counts()
educ_data['currently_enrolled'] = np.where(educ_data.sc1q01 == 3, 'yes', 'no')
educ_data.currently_enrolled.value_counts()

no     599579
yes    214697
Name: currently_enrolled, dtype: int64

In [8]:
C_relevant = educ_data[
    [
        "hhcode",
        "idc",
        "ever_admitted",
        "currently_enrolled",
        "region",
        "province"
    ]
].copy()

C_relevant.sample(5)

,hhcode,idc,ever_admitted,currently_enrolled,region,province
115864,123101222,5,yes,yes,1,1
440203,226100802,6,yes,no,1,2
481765,230103316,2,yes,yes,1,2
318534,214200603,2,no,no,2,2
17230,104101217,4,yes,no,1,1


In [9]:
# MERGE 
years19_20 = pd.merge(B_relevant, C_relevant, on=["hhcode", 'idc'], how='inner', indicator=False)
# Evaluate merge by checking value counts for currently enrolled individuals
years19_20.currently_enrolled.value_counts()

no     599579
yes    214697
Name: currently_enrolled, dtype: int64

In [10]:
# Select people in school enrollment age (I would say 4-10 years old)
years19_20 = years19_20.loc[(years19_20["age"] >= 4)&(years19_20["age"] <= 10)]


In [11]:
years19_20.province.value_counts()

2    81164
1    41991
3    39367
4    20352
Name: province, dtype: int64

In [12]:
# Re-spell the names
years19_20['province'].replace(2, 'Punjab', inplace = True)
years19_20['province'].replace(1, 'KPK', inplace = True)
years19_20['province'].replace(3, 'Sindh', inplace = True)
years19_20['province'].replace(4, 'Balochistan', inplace = True)
years19_20.province.value_counts()

Punjab         81164
KPK            41991
Sindh          39367
Balochistan    20352
Name: province, dtype: int64

In [13]:
# Lets see which hh codes we have:
years19_20["hhcode"] = years19_20["hhcode"].astype("str")
years19_20['subprovince code']= years19_20['hhcode'].apply(lambda x: x[0:4])
years19_20["subprovince code"] = years19_20["subprovince code"].astype("int")
years19_20.head()

,hhcode,idc,age,sex,marital_status,ever_admitted,currently_enrolled,region,province,subprovince code
30235,202100124,1,10,female,unmarried / never married,yes,yes,1,Punjab,2021
48183,209105225,1,10,female,unmarried / never married,yes,yes,1,Punjab,2091
89276,229200805,1,10,male,unmarried / never married,yes,yes,2,Punjab,2292
155736,425100319,1,5,female,unmarried / never married,yes,yes,1,Balochistan,4251
160018,101100115,2,10,male,unmarried / never married,yes,yes,1,KPK,1011


In [14]:
# Add stratum data
stratum_1920 = pd.read_csv('/Users/preetkhowaja/Documents/midssp2022/unifying/uds-2022-ids-701-team-3/00_source_data/stratum1920.csv')
stratum_1920.sample(2)

,subprovince,subprovince code
56,Nankana Sahib,2252
61,Rajanpur,2302


In [15]:
# Merge stratum data with edu and sex data
years19_20_ = pd.merge(years19_20, stratum_1920, on=["subprovince code"], how='inner', indicator=True)

# Evaluate merge 
years19_20_._merge.value_counts()

both          182874
left_only          0
right_only         0
Name: _merge, dtype: int64

In [16]:
years19_20_.drop(columns= ['_merge'], inplace = True)
years19_20_.head()

,hhcode,idc,age,sex,marital_status,ever_admitted,currently_enrolled,region,province,subprovince code,subprovince
0,202100124,1,10,female,unmarried / never married,yes,yes,1,Punjab,2021,Bahawalnagar
1,202100214,2,9,male,unmarried / never married,yes,yes,1,Punjab,2021,Bahawalnagar
2,202100226,2,6,male,unmarried / never married,yes,yes,1,Punjab,2021,Bahawalnagar
3,202100312,2,6,male,unmarried / never married,no,no,1,Punjab,2021,Bahawalnagar
4,202100516,2,7,male,unmarried / never married,yes,yes,1,Punjab,2021,Bahawalnagar


In [17]:
# Indicator for year
years19_20_['Year'] = 2019


In [18]:
years19_20_.marital_status.value_counts()

unmarried / never married                         182873
divorced                                               1
0                                                      0
currently married                                      0
widow / widower                                        0
separated                                              0
nikkah solemnized but rukhsati not taken place         0
Name: marital_status, dtype: int64

In [19]:
# Recode marital status
years19_20_.marital_status.replace('unmarried / never married', 1, inplace = True)
years19_20_.marital_status.replace('currently married', 2, inplace = True)
years19_20_.marital_status.replace('widow / widower', 3, inplace = True)
years19_20_.marital_status.replace('separated', 4, inplace = True)
years19_20_.marital_status.replace('divorced', 4, inplace = True)
years19_20_.marital_status.replace('nikkah solemnized but rukhsati not taken place', 5, inplace = True)
years19_20_.marital_status.value_counts()

1    182873
4         1
0         0
2         0
3         0
5         0
Name: marital_status, dtype: int64

In [20]:
years19_20_.marital_status = years19_20_.marital_status.astype('int')
years19_20_.region.value_counts()

1    136566
2     46308
Name: region, dtype: int64

In [21]:
#Re-code region
years19_20_.region.replace(1, 'rural', inplace = True)
years19_20_.region.replace(2, 'urban', inplace = True)


In [22]:
years19_20_.dtypes

hhcode                  object
idc                       int8
age                      int16
sex                   category
marital_status           int64
ever_admitted           object
currently_enrolled      object
region                  object
province                object
subprovince code         int64
subprovince             object
Year                     int64
dtype: object

In [23]:
# Save to csv
years19_20_.to_csv('/Users/preetkhowaja/Documents/midssp2022/unifying/uds-2022-ids-701-team-3/10_data_cleaning/clean_preet/years19_20_.csv', index_label='id')

In [24]:
years19_20_.sex.value_counts()

male      97552
female    85322
0             0
Name: sex, dtype: int64

In [25]:
years19_20_['sex_new'] = np.where(years19_20_.sex == 'female', 'female', 'male')

In [26]:
years19_20_.sex_new.value_counts()

male      97552
female    85322
Name: sex_new, dtype: int64

In [27]:
years19_20_.sex.value_counts()

male      97552
female    85322
0             0
Name: sex, dtype: int64

In [28]:
# PRE-PROCESSING FOR DIFF-IN-DIFF DATA
# convert currently_enrolled from string to integer
years19_20_["currently_enrolled"].replace("yes", 1, inplace=True)
years19_20_["currently_enrolled"].replace("no", 0, inplace=True)

#check region for anomalies
# years19_20_.region.value_counts()
# years19_20_["region"].replace(3, "urban", inplace=True)
# aggregate dataset for sample population
df_grp_1 = years19_20_.groupby(["sex_new","subprovince","region"])["currently_enrolled"].count().reset_index()
# aggregate dataset for enrollment total
df_grp_2 = years19_20_.groupby(["sex_new","subprovince","region"])["currently_enrolled"].sum().reset_index()
# merge data set
df_grp_merge = pd.merge(df_grp_1, df_grp_2, on=["sex_new","subprovince","region"], indicator=True)
# check merge
df_grp_merge._merge.value_counts()

both          462
left_only       0
right_only      0
Name: _merge, dtype: int64

In [29]:
df_grp_merge.sex_new.value_counts()

female    231
male      231
Name: sex_new, dtype: int64

In [30]:
df_grp_merge.rename(columns = {'sex_new': 'sex'}, inplace=True)
df_grp_merge.head()

,sex,subprovince,region,currently_enrolled_x,currently_enrolled_y,_merge
0,female,Abbottabad,rural,371,318,both
1,female,Abbottabad,urban,48,43,both
2,female,Attock,rural,449,366,both
3,female,Attock,urban,144,112,both
4,female,Awaran,rural,153,47,both


In [31]:
df_grp_merge.sex.value_counts()

female    231
male      231
Name: sex, dtype: int64

In [32]:
# Code Cell 2:
# compute enrollment_rate
df_grp_merge["rate_enrollment"] = (
    df_grp_merge["currently_enrolled_y"] / df_grp_merge["currently_enrolled_x"]
)
# rename columns
df_grp_merge = df_grp_merge.rename(
    columns={
        "currently_enrolled_x": "sample_population",
        "currently_enrolled_y": "enrolled_total",
    }
)
# drop "_merge" column
df_grp_merge.drop(["_merge"], axis=1, inplace=True)
df_grp_merge.sample(5)


,sex,subprovince,region,sample_population,enrolled_total,rate_enrollment
205,female,Swabi,urban,97,68,0.701031
61,female,Jacobabad,rural,465,91,0.195699
96,female,Khushab,urban,135,105,0.777778
398,male,Qilla Saifullah,rural,726,382,0.526171
348,male,Lodhran,urban,113,89,0.787611


In [33]:
# Average rate of enrolment for females in 2019-2020 data
df_grp_merge['rate_enrollment'].loc[df_grp_merge.sex == 'female'].mean()

0.5520207911057974

In [34]:
# Average rate of enrolment for females in 2019-2020 data
df_grp_merge['rate_enrollment'].loc[df_grp_merge.sex == 'male'].mean()

0.6404918770781584

In [35]:
#Code Cell 3:
# save clean difference-in-difference data set
df_grp_merge.to_csv("/Users/preetkhowaja/Documents/midssp2022/unifying/uds-2022-ids-701-team-3/20_analysis/enrollment_clean/years19_20_merge_diff.csv")
